#### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.  

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

#### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

#### Важно:

* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

#### Задание

1.
На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

2.
Реализуйте два алгоритма рекомендаций:
* сортировка просмотренных id по популярности (частота появления в просмотренных),
* сортировка просмотренных id по покупаемости (частота появления в покупках).


3.
Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие?           Обратите внимание на различие качества на обучающей и тестовой           выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)



#### Дополнительные вопросы

1.
Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.

2.
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [1]:
import numpy as np
import collections
import pandas as pd

In [2]:
# считываем построчно и разносим отдельно просмотренные id от купленных
# все складываем в словарь
def load_data(file):
    f = open(file)
    data = {}
    file_len = len(f.readlines())
    f.seek(0)
    for i in range(file_len):
        line = list(f.readline().rstrip('\n').split(';'))
        view = list(line[0].split(','))
        view = list(map(lambda x: int(x), view))
        if line[1] != '':
            pur = list(line[1].split(','))
            pur = list(map(lambda x: int(x), pur))
        #    data[i] = (view, pur)
        else:
            pur = np.nan
        data[i] = (view, pur)
    f.close()
    return data

In [3]:
train_data = load_data('coursera_sessions_train.txt')
test_data = load_data('coursera_sessions_test.txt')

In [4]:
train_df = pd.DataFrame.from_dict(train_data, orient = 'index', columns = ['view', 'purch'])
test_df = pd.DataFrame.from_dict(test_data, orient = 'index', columns = ['view', 'purch'])

In [5]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [6]:
train_df.head(10)

,view,purch
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86]
19,"[138, 198, 199, 127]",[199]
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303]
33,"[352, 353, 352]",[352]
55,[519],[519]
64,"[599, 600, 601, 602]","[603, 604, 602, 599, 605, 606, 600]"
72,"[687, 688, 689, 690, 691, 690, 688, 690, 688, ...","[690, 688]"
89,"[850, 851, 852]",[851]
93,"[879, 884, 170, 137, 170, 879, 884, 879, 885, ...",[879]


In [7]:
list(test_df.view.iloc[0:4])

[[63, 68, 69, 70, 66, 61, 59, 61, 66, 68],
 [158, 159, 160, 159, 161, 162],
 [200, 201, 202, 203, 204],
 [371, 372, 371]]

In [8]:
# функция подсчета повторений id в просмотренных или купленных
def counter(dict_data, is_view):
    freq_count = collections.Counter()

    if is_view:
        rank = 0
    else:
        rank = 1
        
    for i in range(len(dict_data)):
        if dict_data[i][rank] is not np.nan:
            for j in dict_data[i][rank]:
                freq_count[j] += 1

    return freq_count

In [9]:
# считаем количество просмотров и покупок по каждому id
train_view = counter(train_data, True)
train_purch = counter(train_data, False)

In [33]:
# функция рассчета метрик для всех вариантов данных
def Scores(data, view_purch = 0):
    """
    если view_purch = 0, то считаем по частоте просмотров
    если view_purch = 1, то считаем по частоте покупок
    """
    AvgPr5 = []
    AvgPr1 = []
    AvgRe5 = []
    AvgRe1 = []
    for i in range(len(data)):
        temp_df = pd.DataFrame(columns = ['freq_v', 'freq_p', 'pos', 'id'])

        for p, j in enumerate(data.view.iloc[i]):
            item = j
            pos = p
            fr_v = train_view[j]
            fr_p = train_purch[j]
            temp_df.loc[p] = [fr_v, fr_p, pos, item]
        
        temp_df = temp_df.drop_duplicates('id')
        
        if view_purch:
            temp_df.sort_values(['freq_p', 'pos'], ascending = [False, True], inplace = True)
        else:
            temp_df.sort_values(['freq_v', 'pos'], ascending = [False, True], inplace = True)
        
        Pr5 = len(set(temp_df.id.iloc[0:5]).intersection(data.purch.iloc[i]))
        Pr1 = len(set(temp_df.id.iloc[0:1]).intersection(data.purch.iloc[i]))
        
        Re5 = Pr5 / len(data.purch.iloc[i])
        Re1 = Pr1 / len(data.purch.iloc[i])
        
        Pr5 = Pr5 / 5
                  
        AvgPr5.append(Pr5)
        AvgPr1.append(Pr1)
        AvgRe5.append(Re5)
        AvgRe1.append(Re1)    
        
    return round(np.mean(AvgRe1),2), round(np.mean(AvgPr1),2), round(np.mean(AvgRe5),2), round(np.mean(AvgPr5),2)

In [14]:
def save_answer(file_name, answer):
    with open(file_name, "w") as f:
        f.write(" ".join([str(x) for x in answer]))

In [30]:
save_answer('views_popularity_train.txt', Scores(train_df, 0))

In [21]:
save_answer('views_popularity_test.txt' ,Scores(test_df, 0))

In [17]:
save_answer('purchases_popularity_train.txt', Scores(train_df, 1))

In [18]:
save_answer('purchases_popularity_test.txt', Scores(test_df, 1))